In [1]:
!cp drive/MyDrive/MusicGenre/KRF.py .
!cp drive/MyDrive/MusicGenre/metrics.py .
!cp drive/MyDrive/MusicGenre/NeuralNetwork.py .
!cp drive/MyDrive/MusicGenre/GCN.py .
# !cp -av drive/MyDrive/MusicGenre/data .
!cp -av drive/MyDrive/MusicGenre/glove42b .
!cp -av drive/MyDrive/MusicGenre/checkpoints .


'drive/MyDrive/MusicGenre/glove42b' -> './glove42b'
'drive/MyDrive/MusicGenre/glove42b/amazon_music' -> './glove42b/amazon_music'
'drive/MyDrive/MusicGenre/glove42b/amazon_music/adj_file.pkl' -> './glove42b/amazon_music/adj_file.pkl'
'drive/MyDrive/MusicGenre/glove42b/amazon_music/label_dic.pkl' -> './glove42b/amazon_music/label_dic.pkl'
'drive/MyDrive/MusicGenre/glove42b/amazon_music/X_train.pkl' -> './glove42b/amazon_music/X_train.pkl'
'drive/MyDrive/MusicGenre/glove42b/amazon_music/X_test.pkl' -> './glove42b/amazon_music/X_test.pkl'
'drive/MyDrive/MusicGenre/glove42b/amazon_music/X_valid.pkl' -> './glove42b/amazon_music/X_valid.pkl'
'drive/MyDrive/MusicGenre/checkpoints' -> './checkpoints'


In [2]:
import pickle
import torch
from metrics import Metrics
from sklearn.metrics import f1_score
import numpy as np
from KRF import KRF
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [3]:
def loadData(task):
    # path = "./data/" + task + "/"
    path = "./glove42b/" + task + "/"
    X_train_label, X_train_comment,  word_embeddings = pickle.load(open(path + "X_train.pkl", mode='rb'))
    X_valid_label, X_valid_comment = pickle.load(open(path + "X_valid.pkl", mode='rb'))
    X_test_label, X_test_comment = pickle.load(open(path + "X_test.pkl", mode='rb'))

    config["C"] = 22 if task == 'douban_music' else 20
    config["task"] = task
    config['adj_file'] = path + "adj_file.pkl"
    config['max_comments'] = len(X_train_comment[0])
    config['max_len'] = len(X_train_comment[0][0])
    config['embedding_weights'] = word_embeddings

    return  X_train_label, X_train_comment,  \
            X_valid_label, X_valid_comment,  \
            X_test_label, X_test_comment


def train_and_test(model_class, task):
    model_suffix = model_class.__name__.lower().strip("text")
    config['save_path'] = 'checkpoints/weights.best.'+ model_suffix

    X_train_label, X_train_comment, \
    X_valid_label, X_valid_comment, \
    X_test_label, X_test_comment = loadData(task)

    model = model_class(config)
    model.fit(X_train_comment, X_train_label,
              X_valid_comment, X_valid_label)

    print("================================================")
    model.load_state_dict(state_dict=torch.load(config['save_path']))
    y_pred, y_pred_top = model.predict(X_test_comment)

    metric = Metrics()
    metric.calculate_all_metrics(X_test_label, y_pred, y_pred_top)

config = {
    'lr':1e-3,
    'reg':0,
    'alpha':1.0,
    'batch_size':128,
    'maxlen':1000,
    'nb_filters': 100,
    'dropout': 0.5,
    'input_dim':300,
    'epochs':20,
}


In [4]:
if __name__ == '__main__':
    model = KRF
    train_and_test(model, task="amazon_music")

KRF(
  (word_embedding): Embedding(49793, 300, padding_idx=0)
  (label_embedding): Embedding(20, 200)
  (word_bigru): GRU(300, 100, batch_first=True, bidirectional=True)
  (sent_bigru): GRU(200, 100, batch_first=True, bidirectional=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (word_attention): Attention(
    (dropout): Dropout(p=0.5, inplace=False)
    (linear1): Linear(in_features=200, out_features=100, bias=True)
    (linear2): Linear(in_features=100, out_features=1, bias=True)
  )
  (sent_attention): Attention(
    (dropout): Dropout(p=0.5, inplace=False)
    (linear1): Linear(in_features=200, out_features=100, bias=True)
    (linear2): Linear(in_features=100, out_features=1, bias=True)
  )
  (fc1): Linear(in_features=200, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=20, bias=True)
  (gcn): RelationNet(
    (gc1): GraphConvolution (200 -> 512)
    (gc2): GraphConvolution (512 -> 100)
    (relu): LeakyReLU(negative_slope=0.2)
  )
)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Batch[0] - loss: 0.668904
Batch[1] - loss: 0.490305
Batch[2] - loss: 0.527388
Batch[3] - loss: 0.455594
Batch[4] - loss: 0.469858
Batch[5] - loss: 0.442421
Batch[6] - loss: 0.420477
Batch[7] - loss: 0.451628
Batch[8] - loss: 0.423511
Batch[9] - loss: 0.416530
Batch[10] - loss: 0.426129
Batch[11] - loss: 0.413322
Batch[12] - loss: 0.385532
Batch[13] - loss: 0.386773
Batch[14] - loss: 0.421744
Batch[15] - loss: 0.385031
Batch[16] - loss: 0.374903
Batch[17] - loss: 0.385578
Batch[18] - loss: 0.369269
Batch[19] - loss: 0.368217
Batch[20] - loss: 0.352185
Batch[21] - loss: 0.351449
Batch[22] - loss: 0.360929
Batch[23] - loss: 0.352002
Batch[24] - loss: 0.348137
Batch[25] - loss: 0.350066
Batch[26] - loss: 0.346908
Batch[27] - loss: 0.352199
Batch[28] - loss: 0.353948
Batch[29] - loss: 0.344032
Batch[30] - loss: 0.360697
Batch[31] - loss: 0.334059
Batch[32] - loss: 0.347901
Batch[33] - loss: 0.347912
Batch[34] - loss: 0.357601
Batch[35] - loss: 0.344676
Batch[36] - loss: 0.339513
Batch[37] -

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Batch[52] - loss: 0.311948
Average loss:0.378296 average acc:0.000000%
OE:  0.37273991655076494
HL:  0.12054937413073713
MacroF1:  0.04064145524174246
MicroF1:  0.32875121006776375
Val set metric: -0.07990791888899468
Best val set metric: 0

Epoch  2 / 20
Batch[0] - loss: 0.326363
Batch[1] - loss: 0.358704


...

Batch[1] - loss: 0.027595
Batch[2] - loss: 0.028012
Batch[3] - loss: 0.023501
Batch[4] - loss: 0.025683
Batch[5] - loss: 0.036948
Batch[6] - loss: 0.017861
Batch[7] - loss: 0.039520
Batch[8] - loss: 0.030271
Batch[9] - loss: 0.026402
Batch[10] - loss: 0.037176
Batch[11] - loss: 0.032011
Batch[12] - loss: 0.029274
Batch[13] - loss: 0.017722
Batch[14] - loss: 0.026511
Batch[15] - loss: 0.029875
Batch[16] - loss: 0.031861
Batch[17] - loss: 0.030261
Batch[18] - loss: 0.024394
Batch[19] - loss: 0.032378
Batch[20] - loss: 0.027287
Batch[21] - loss: 0.033095
Batch[22] - loss: 0.031212
Batch[23] - loss: 0.029148
Batch[24] - loss: 0.031344
Batch[25] - loss: 0.034180
Batch[26] - loss: 0.032618
Batch[27] - loss: 0.032532
Batch[28] - loss: 0.029520
Batch[29] - loss: 0.036557
Batch[30] - loss: 0.037262
Batch[31] - loss: 0.037501
Batch[32] - loss: 0.034595
Batch[33] - loss: 0.023126
Batch[34] - loss: 0.022461
Batch[35] - loss: 0.028980
Batch[36] - loss: 0.030517
Batch[37] - loss: 0.026041
Batch[38] 

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Batch[51] - loss: 0.041952
Batch[52] - loss: 0.031374
Average loss:0.029259 average acc:0.000000%
OE:  0.0952712100139082
HL:  0.06383866481223922
MacroF1:  0.6610694676187333
MicroF1:  0.7592446892210857
Val set metric: 0.5972308028064941
Best val set metric: 0.6013577418228009
Now lr:  7.8125e-06
Decay learning rate to:  7.8125e-06
Reload the best model...


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


OE:  0.07440890125173853
HL:  0.06460361613351878
MacroF1:  0.6514851425733033
MicroF1:  0.7542328042328044
